In [29]:
#pip install easyAI
#from easyAI import TwoPlayersGame, Human_Player, AI_Player, Negamax
import tkinter
from tkinter import ttk
from tkinter import *

from easyAI import TwoPlayersGame
from easyAI.Player import Human_Player

from neo4j import GraphDatabase

In [31]:
#ll = []
#CLASE PAR CREAR NODO CENTRAR-PARQUE CENTRAL
# class CLASE_NEO4J(object): 
#     def __init__(self):
#         self._driver = GraphDatabase.driver("bolt:neo4j://localhost:7687", auth=("neo4j", "neo4jj"), encrypted=False)
#     def close(self):
#         self._driver.close()
driver = GraphDatabase.driver("bolt:neo4j://localhost:7687", auth=("neo4j", "neo4jj"), encrypted=False)
session1 = driver.session(database="neo4j")
session1 = driver.session()
#     def OBTENER_TIPO(self,ti):
#         with self._driver.session() as session:
#             greeting = session.write_transaction(self._VALIDAR_LUGAR, ti)
#             print(greeting)
            
#     @staticmethod
#     def _VALIDAR_LUGAR(tx, ti):
#         #SE BUSCA SI EL LUGAR DEL ARREGLO EXISTE EN LA BASE NEO4J
#         result2 = tx.run("Match (n:Lugares) where n.tipo='"+ti+"' return n.nombre AS nombre").data()
#         ll = []
#         for h in result2:
#             ll.append(h)
# #             print(h)
# #             print(type(h["nombre"]))
        
            
# grafo=CLASE_NEO4J()

In [100]:
class TicTacToe( TwoPlayersGame ):
    """ The board positions are numbered as follows:
            1 2 3
            4 5 6
            7 8 9 
    """    

    def __init__(self, players):
        self.players = players
        self.board = [0 for i in range(9)]
        self.nplayer = 1 # player 1 starts.
        
    def possible_moves(self):
        return [i+1 for i,e in enumerate(self.board) if e==0]
    
    def make_move(self, move):
        self.board[int(move)-1] = self.nplayer

    def unmake_move(self, move): # optional method (speeds up the AI)
        self.board[int(move)-1] = 0
    
    def lose(self):
        """ Has the opponent "three in line ?" """
        return any( [all([(self.board[c-1]== self.nopponent)
                      for c in line])
                      for line in [[1,2,3],[4,5,6],[7,8,9], # horiz.
                                   [1,4,7],[2,5,8],[3,6,9], # vertical
                                   [1,5,9],[3,5,7]]]) # diagonal
        
    def is_over(self):
        return (self.possible_moves() == []) or self.lose()
        
    def show(self):
        a = '\n'+'\n'.join([
                     ' '.join([['.','O','X'][self.board[3*j+i]]
                     for i in range(3)])
              for j in range(3)]) 
        
        #return a
        print(a)
        #print(type(a))
#         print ('\n'+'\n'.join([
#                         ' '.join([['.','O','X'][self.board[3*j+i]]
#                         for i in range(3)])
#                  for j in range(3)]) )
    def scoring(self):
        return -100 if self.lose() else 0
    
if __name__ == "__main__":
  
    from easyAI import AI_Player, Negamax
    def inicio():
        ai_algo = Negamax(6)
        aa = TicTacToe( [Human_Player(),AI_Player(ai_algo)]).play()
        lbl = Label(window, text=aa)
        print(aa)
        
    lh = []
    lr = []
    llt = []
    def obtenerH(ho):  
        #query = "[CALL gds.graph.create('k1Localidad',{Restaurant : {label: 'Restaurant',properties: 'gn'}},'*')]"
        query =session1.run("Match (n:Lugares) where n.tipo='"+ho+"' return n.nombre AS nombre") 
        for h in query:
            lh.append(h['nombre'])
        return lh
    
    def obtenerR(re):   
        #query = "[CALL gds.graph.create('k1Localidad',{Restaurant : {label: 'Restaurant',properties: 'gn'}},'*')]"
        query =session1.run("Match (n:Lugares) where n.tipo='"+re+"' return n.nombre AS nombre ") 
        for h in query:
            lr.append(h['nombre'])
        return lr
    
    def obtenerLT(lt):   
        #query = "[CALL gds.graph.create('k1Localidad',{Restaurant : {label: 'Restaurant',properties: 'gn'}},'*')]"
        query =session1.run("Match (n:Lugares) where n.tipo='"+lt+"' return n.nombre AS nombre ") 
        for h in query:
            llt.append(h['nombre'])
        return llt
    
    def sim():
        query = session1.run("CALL gds.beta.knn.stream('myGraph2', {topK: 1,nodeWeightProperty: 'latitud',randomSeed: 42,concurrency: 1,sampleRate: 1.0,deltaThreshold: 0.0})YIELD node1, node2, similarity RETURN gds.util.asNode(node1).nombre AS Lugar1, gds.util.asNode(node2).nombre AS Lugar2, similarity ORDER BY similarity DESCENDING, Lugar1, Lugar2")
        lknn = []
        for s in query:
            dict_knn = {
              "lugar1": s['Lugar1'],
              "lugar2": s['Lugar2'],
              "similitud": s['similarity']
            }
            lknn.append(dict_knn)
        return lknn
    def limpiar():
        listbox.delete(0,END)
        listbox2.delete(0,END)
    def ob():
        listbox2.delete(0,END)
        a= listbox.get(listbox.curselection()[0])
        hj= []
        for i in range(0,43):
            l1 = sim()[i]['lugar1'] if a == sim()[i]['lugar2'] else ""
            l2 = sim()[i]['lugar2'] if a == sim()[i]['lugar1'] else ""
            hj.append(l1)
            hj.append(l2)
        hg = list(set(hj))
        hg.pop(0)
        listbox2.insert(0, *hg)
        #return hg
 
    def pri():
        if (combo.get() == "Hotel"):
            hh = obtenerH(combo.get())
            listbox.insert(0, *hh)
        elif (combo.get() == "Restaurante"):
            hh1 = obtenerR(combo.get())
            listbox.insert(0, *hh1)
        elif (combo.get() == "Lugar_Turistico"):
            hh2 = obtenerLT(combo.get())
            listbox.insert(0, *hh2)
            
    global window
    window = tkinter.Tk()
    window.title("Welcome to LikeGeeks app")
    window.geometry('750x750')

    nb = ttk.Notebook(window)
    nb.pack(fill='both', expand='yes')
    
    pl = ttk.Frame(nb)
    pll = ttk.Frame(nb)
    
    combo = ttk.Combobox(pll)
    combo.place(x=50, y=50)
    combo["values"] = ["Hotel", "Restaurante", "Lugar_Turistico"]
    
    listbox = tkinter.Listbox(pll)
    listbox.grid(row=4, column=3)
    
    listbox2 = tkinter.Listbox(pll)
    listbox2.grid(row=4, column=9)
    
    Button(pl, text="Jugar", command=inicio).place(x=220, y=160)
    Button(pll, text="Recomendar", command=pri).place(x=120, y=160)
    Button(pll, text="Limpiar", command=limpiar).place(x=380, y=300)
    Button(pll, text="Mostrar Lugar Cercano", command=ob).place(x=250, y=250)

    lbl = Label(pll, text="Que le gustaria que le recomienden?")
    lbl.grid(column=0, row=0)

    nb.add(pl, text='Pestaña1')
    nb.add(pll, text='Pestaña2')

    window.mainloop()